In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# CNN

In [3]:
import os
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import shuffle
import seaborn as sn; sn.set(font_scale=1.4)
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D


In [4]:
# Define the folders containing the videos
extrovert_folder = '/content/drive/MyDrive/PFA/videos/train/extrovert'
introvert_folder = '/content/drive/MyDrive/PFA/videos/train/introvert'

In [5]:
# Load the videos and extract the frames
frames = []
labels = []
for folder, label in [(extrovert_folder, 0), (introvert_folder, 1)]:
    for filename in os.listdir(folder):
        if filename.endswith('.mp4'):
            video_path = os.path.join(folder, filename)
            cap = cv2.VideoCapture(video_path)
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (224, 224))
                frames.append(frame)
                labels.append(label)
            cap.release()
frames

[array([[[196, 177, 151],
         [145, 126, 100],
         [191, 172, 146],
         ...,
         [231, 189, 137],
         [228, 187, 131],
         [228, 187, 131]],
 
        [[169, 150, 124],
         [136, 117,  91],
         [192, 173, 147],
         ...,
         [231, 189, 137],
         [228, 187, 131],
         [228, 187, 131]],
 
        [[136, 117,  91],
         [136, 117,  91],
         [191, 171, 145],
         ...,
         [231, 189, 137],
         [231, 190, 134],
         [231, 190, 134]],
 
        ...,
 
        [[ 25,  27,  34],
         [ 82,  91,  97],
         [137, 156, 160],
         ...,
         [ 87, 134, 133],
         [151, 184, 174],
         [107, 140, 130]],
 
        [[ 46,  59,  67],
         [ 75,  88,  96],
         [ 98, 111, 120],
         ...,
         [ 57,  98, 101],
         [133, 165, 160],
         [120, 153, 147]],
 
        [[ 28,  41,  49],
         [ 95, 108, 116],
         [108, 121, 129],
         ...,
         [ 48,  88,  91],
  

In [6]:
# Convert the frames to a numpy array and preprocess them
frames = np.array(frames, dtype=np.float32) / 255.0
frames -= np.array([0.485, 0.456, 0.406])
frames /= np.array([0.229, 0.224, 0.225])

In [7]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Define a simple CNN model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [9]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# Split the data into training and validation sets
num_samples = len(frames)
split_idx = int(0.8 * num_samples)
train_frames, train_labels = frames[:split_idx], labels[:split_idx]
val_frames, val_labels = frames[split_idx:], labels[split_idx:]
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [ ]:
# Train the model
history = model.fit(train_frames, train_labels, epochs=5, batch_size=32, validation_data=(val_frames, val_labels))

Epoch 1/5
111/111 [==============================] - 8325s 75s/step - loss: 0.6390 - accuracy: 0.7646 - val_loss: 1.5828 - val_accuracy: 0.0000e+00
Epoch 2/5
111/111 [==============================] - 8184s 74s/step - loss: 0.5482 - accuracy: 0.7734 - val_loss: 1.6389 - val_accuracy: 0.0000e+00
Epoch 3/5
 73/111 [==================>...........] - ETA: 43:51 - loss: 0.5342 - accuracy: 0.7812

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(val_frames, val_labels)
print('Test accuracy:', test_acc)